#### Model integration with GROQ and Google Gemini

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

#### Google Gemini Model integration

In [18]:
from langchain.chat_models import init_chat_model ## To initialize any model

model1 = init_chat_model("google_genai:gemini-2.5-flash-lite")
response = model1.invoke("Give one sentence about India.")

response.content

'India is a vibrant and diverse subcontinent known for its ancient history, rich cultural tapestry, and rapid economic growth.'

#### Model integration with Groq

In [36]:
from langchain_groq import ChatGroq

model = init_chat_model("groq:qwen/qwen3-32b")
#model = ChatGroq(model="llama-3.1-8b-instant")
response = model.invoke("One word about Bengaluru")

response.content

'<think>\nOkay, the user is asking for one word about Bengaluru. Let me think. First, I need to recall what I know about Bengaluru. It\'s the capital of Karnataka, India. Known for its tech industry, right? So maybe "tech hub" or "IT city" come to mind. But the user wants a single word.\n\nHmm, "Silicon Valley of India" is a term often used for Bengaluru. That\'s a common nickname. But is that one word? Well, "Silicon Valley" is two words, but maybe the user expects that as a single term. Alternatively, maybe "Garden City" because it\'s a historical nickname referring to its greenery. But that\'s also two words. Wait, the user specified "one word," so maybe I should pick one of those two nicknames.\n\nAnother angle: Bengaluru is known for its weather, which is pleasant compared to other Indian cities. So "pleasant" or "cool" could be words, but those are not unique to Bengaluru. The tech angle is more distinctive. "Innovation" is a word, but maybe too generic. "Startup" is another poss

#### Streaming

In [ ]:
for chunk in model.stream("Write a 200n words paragraph about Mother"):
    print(chunk.text)

#### Batch

In [10]:
responses = model.batch([
    "Give sentence about India",
    "Explain Artificial intelligence"
],
config={
    'max_concurrency': 5
}
)

responses


[AIMessage(content="India is the world's largest democracy and the seventh-largest country by land area, with a rich cultural heritage and a diverse geography that includes the Himalayan Mountains, the Indo-Gangetic Plain, and the Deccan Plateau.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 39, 'total_tokens': 87, 'completion_time': 0.097347036, 'completion_tokens_details': None, 'prompt_time': 0.00778374, 'prompt_tokens_details': None, 'queue_time': 0.055464259, 'total_time': 0.105130776}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bdfaf-b4bd-7d40-96bb-1de55d45a36f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 39, 'output_tokens': 48, 'total_tokens': 87}),
 AIMessage(content='**Artificial Intelligence (AI): A Comprehensive Overview**\n\nArtificial Intellige

#### Tools -> 

Models can use this tools to perform tasks such as fetching data from database, searching the web, or running code.

In [ ]:
from langchain_core.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"The weather in {location} is cloudy"

model_with_tools = model.bind(tools = [get_weather])

response = model_with_tools.invoke("What's the weather in Mandya?")
print(response)
for tool_call in response.tool_calls:
    #view tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")    


In [ ]:
#Step 1: Model generates tool calls
messages = [{"role": "user", "content": "Whats the weather in Singapore?"}]
ai_msg = model_with_tools.invoke(messages)

print(ai_msg)

#Step2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

print(messages)

#Step3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response)
print(final_response.text)
